https://github.com/miladlink/TinyYoloV2

# Libraries

In [1]:

# import os
from PIL import Image
import numpy as np
import json
import cv2
from tqdm import tqdm
# import skimage.io as io
# import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import torch
import torchvision
from torchvision import transforms
import torchvision.transforms as transforms
from torchvision.datasets.coco import CocoDetection
from torch.utils.data import DataLoader

from utils.YOLOv2 import *
from models.YOLOv3 import load_model
from attacks.FGSM import FGSM
from detect import detect_image
from utils.loss import compute_loss
from utils.utils import load_classes, rescale_boxes, non_max_suppression, print_environment_info
from utils.augmentations import TRANSFORM_TRAIN, TRANSFORM_VAL
from utils.transforms import DEFAULT_TRANSFORMS, Resize, ResizeEval

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Model

In [3]:
modelv = 3
img_size=416

if modelv == 2:
    model = load_model_v2(weights = './weights/yolov2-tiny-voc.weights').to(device)
    class_names = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'TVmonitor'] 
    root_train = "./data/VOC2007/JPEGImages"
    annFile_train = "./data/VOC2007/annotations/train.json"
    root_val = "./data/VOC2007/JPEGImages"
    annFile_val = "./data/VOC2007/annotations/val.json"
    
elif modelv == 3:
    model = load_model("./config/yolov3.cfg", "./weights/yolov3.weights")
    class_names = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
    id_list = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90])
    root_train = "./data/COCO2017/train2017"
    annFile_train = "./data/COCO2017/annotations/instances_train2017_modified.json"
    root_val = "./data/COCO2017/val2017"
    annFile_val = "./data/COCO2017/annotations/instances_val2017_modified.json"
    
else:
    print("invalid model number!")

# COCO loader

create dataloader (make different train and val later)

In [4]:
# coco_dataset_train = CocoDetection(root=root_train, annFile=annFile_train, transform=TRANSFORM_TRAIN_IMG, target_transform=TRANSFORM_TRAIN_TARGET)
coco_dataset_val = CocoDetection(root=root_val, annFile=annFile_val, transforms=TRANSFORM_VAL)
coco_dataset_eval = CocoDetection(root=root_val, annFile=annFile_val, transform=transforms.Compose([transforms.ToTensor(),]))

def collate_fn(batch):
    return tuple(zip(*batch))

# Create a DataLoader for your COCO dataset
# train_loader = DataLoader(coco_dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(coco_dataset_val, batch_size=1, shuffle=True, collate_fn=collate_fn)
cocoeval_loader = DataLoader(coco_dataset_eval, batch_size=1, shuffle=True, collate_fn=collate_fn)
# images, annotations = next(iter(val_loader))
# np.set_printoptions(linewidth=500)
# np.set_printoptions(suppress=True)
# print("dataloader out")
# print(annotations[0].numpy())

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


helper functions


In [5]:
def xyxy2xywh(x):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[..., 0] = (x[..., 0] + x[..., 2]) / 2  # x center
    y[..., 1] = (x[..., 1] + x[..., 3]) / 2  # y center
    y[..., 2] = x[..., 2] - x[..., 0]  # width
    y[..., 3] = x[..., 3] - x[..., 1]  # height
    return y

def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bottom right y
    return y

# Eval for COCOeval

In [7]:
predictions = []
img_size=416

for i, data in enumerate(tqdm(cocoeval_loader)): # * ASSUME ONE IMAGE PER BATCH FOR NOW
    with torch.no_grad():
        images, targets = data
        # print(targets[0])
        if len(targets[0]) > 0:
            model.eval()
            
            # * getting necessary values
            # for target in targets[0]: print(f"id {target['image_id']}, class {target['category_id']}, bbox {target['bbox']}")
            image_id = targets[0][0]["image_id"]
            category_id = targets[0][0]["category_id"]
            img = images[0].unsqueeze(0).to(device)
            img_copy = img.clone().detach()
            # print(img.shape) # torch.Size([1, 3, 375, 500])
            
            # * turn target into 2d numpy matrix for loss calc
            targetsMatrix = np.zeros((len(targets[0]), 6))
            for i, d in enumerate(targets[0]):
                targetsMatrix[i, 0] = d['image_id']
                targetsMatrix[i, 1] = d['category_id']
                targetsMatrix[i, 2:6] = d['bbox']
            # np.set_printoptions(suppress=True)
            # print(targetsMatrix)

            # * just for sanity check, output image. put the dim 3 at the back
            # imageN = img.clone().detach()
            # imageN = imageN.cpu().squeeze().permute(1, 2, 0).numpy() 
            # imageN = cv2.cvtColor(imageN, cv2.COLOR_RGB2BGR)
            # # print(imageN.shape)
            # cv2.imwrite("output/mygraph.jpg", imageN*255) 

            # * put images into model and get bboxes
            if modelv == 2:
                resized_tensor_image = torch.nn.functional.interpolate(img, size=(416, 416), mode='bilinear', align_corners=False)
                # print(resized_tensor_image.shape)
                # * default 0.66, 0.55, higher conf -> more strict, higher nms -> more iou needed -> more strict
                boxes = filtered_boxes(model, resized_tensor_image, conf_thresh=0.5, nms_thresh=0.9, device=device)
                boxes = np.delete(boxes, 5, axis=1) # x_center, y_center, w, h, bbox_conf, cls_conf, cls -> delete cls_conf
            elif modelv == 3:
                img = img.cpu().squeeze().permute(1, 2, 0).numpy()
                img = transforms.Compose([DEFAULT_TRANSFORMS, ResizeEval(img_size)])((img, np.zeros((1, 5))))[0].unsqueeze(0).to(device)
                boxes = model(img)
                boxes = non_max_suppression(boxes, conf_thres=0.3, iou_thres=0.5)[0].numpy()
                # print(boxes.shape, img_size, og_shape[2:])
                # print(boxes)
                
                boxes = rescale_boxes(boxes, img_size, img_copy.shape[2:]) # rescale back to original proportions?
                boxes = xyxy2xywh(boxes) # convert from coco to yolo
                boxes[:,0] = boxes[:,0]/img_copy.shape[3]
                boxes[:,1] = boxes[:,1]/img_copy.shape[2]
                boxes[:,2] = boxes[:,2]/img_copy.shape[3]
                boxes[:,3] = boxes[:,3]/img_copy.shape[2]
            else: 
                print("invalid model num!")
                
            
            # * another sanity check
            # to_pil = transforms.ToPILImage() 
            # pil_image = to_pil(img_copy.squeeze())
            # pred_img = plot_boxes(pil_image, boxes, None, class_names)
            # pil_image.save("./output/path_to_save_image.jpg")
            
            # * put into coco format of x_min,y_min, width, height, bbox_conf, cls
            # yolo format is x_center, y_center, w, h, bbox_conf, cls_conf, cls
            for box in boxes:
                x_center, y_center, w, h, conf, cls = box
                x_min = max(0, (x_center - w / 2) * img_copy.shape[3])
                y_min = max(0, (y_center - h / 2) * img_copy.shape[2])
                width = min(img_copy.shape[3], w * img_copy.shape[3])
                height = min(img_copy.shape[2], h * img_copy.shape[2])
                # print(x_min,y_min, width, height, bbox_conf, cls)
                predictions.append({
                    'image_id': image_id,
                    'category_id': int(id_list[int(cls)]) if modelv == 3 else int(cls),
                    'bbox': [int(x_min), int(y_min), int(width), int(height)],
                    'score': round(float(conf),2)
                })
            
        else: # pics without labels
            continue
            # print("error", targets) 
    # break

with open(f'./data/results/v{modelv}predictions.json', 'w') as f:
    json.dump(predictions, f)

100%|██████████| 5000/5000 [02:38<00:00, 31.55it/s]


# Eval for loss

inference and output json file containing prediction

In [ ]:
predictions = []

for i, data in enumerate(tqdm(val_loader)): # * ASSUME ONE IMAGE PER BATCH FOR NOW
    with torch.no_grad():
        images, targets = data
        if len(targets[0]) > 0:
            model.eval()
            # print(targets)
            # * getting necessary values
            # for target in targets[0]: print(f"id {target['image_id']}, class {target['category_id']}, bbox {target['bbox']}")
            image_id = int(targets[0][0][0])
            category_id = targets[0][:, 0]
            img = images[0].unsqueeze(0).to(device)
            img_copy = img.clone().detach()
            # print(img.shape) # torch.Size([1, 3, 375, 500])

            # * just for sanity check, output image. put the dim 3 at the back
            # imageN = img.clone().detach()
            # imageN = imageN.cpu().squeeze().permute(1, 2, 0).numpy() 
            # imageN = cv2.cvtColor(imageN, cv2.COLOR_RGB2BGR)
            # # print(imageN.shape)
            # cv2.imwrite("output/mygraph.jpg", imageN*255) 

            # * put images into model and get bboxes
            if modelv == 2:
                resized_tensor_image = torch.nn.functional.interpolate(img, size=(416, 416), mode='bilinear', align_corners=False)
                # print(resized_tensor_image.shape)
                # * default 0.66, 0.55, higher conf -> more strict, higher nms -> more iou needed -> more strict
                boxes = filtered_boxes(model, resized_tensor_image, conf_thresh=0.5, nms_thresh=0.9, device=device)
                if boxes != []: boxes = np.delete(boxes, 5, axis=1) # x_center, y_center, w, h, bbox_conf, cls_conf, cls -> delete cls_conf
            
            elif modelv == 3:
                boxes = model(img)
                boxes = non_max_suppression(boxes, conf_thres=0.3, iou_thres=0.5)[0].numpy()
                boxes = rescale_boxes(boxes, img_size, img_copy.shape[2:]) # rescale back to original, for cocoeval
                boxes = xyxy2xywh(boxes) # nms returns xyxy, convert to yolo
                boxes[:,0] = boxes[:,0]/img_copy.shape[3]
                boxes[:,1] = boxes[:,1]/img_copy.shape[2]
                boxes[:,2] = boxes[:,2]/img_copy.shape[3]
                boxes[:,3] = boxes[:,3]/img_copy.shape[2]
            
                # boxes[:, 5] = id_list[boxes[:, 5].astype(int)] # convert class to category_id 
            else: 
                print("invalid model num!")
                
            
            # # * another sanity check
            # to_pil = transforms.ToPILImage() 
            # pil_image = to_pil(img_copy.squeeze())
            # pred_img = plot_boxes(pil_image, boxes, None, class_names)
            # pil_image.save("./output/path_to_save_image.jpg")
            
            # * put into coco format of x_min,y_min, width, height, bbox_conf, cls
            # yolo format is x_center, y_center, w, h, bbox_conf, cls_conf, cls
            for box in boxes:
                x_center, y_center, w, h, conf, cls = box
                x_min = max(0, (x_center - w / 2) * img_copy.shape[3])
                y_min = max(0, (y_center - h / 2) * img_copy.shape[2])
                width = min(img_copy.shape[3], w * img_copy.shape[3])
                height = min(img_copy.shape[2], h * img_copy.shape[2])
                # print(x_min,y_min, width, height, bbox_conf, cls)
                predictions.append({
                    'image_id': int(image_id),
                    'category_id': int(id_list[int(cls)]) if modelv == 3 else int(cls),
                    'bbox': [int(x_min), int(y_min), int(width), int(height)],
                    'score': round(float(conf),2)
                })
            
        else: # pics without labels
            continue
            # print("error", targets) 
    break
print(predictions)

with open(f'./data/results/v{modelv}predictions.json', 'w') as f:
    json.dump(predictions, f)

  0%|          | 0/5000 [00:00<?, ?it/s]

[{'image_id': 173302, 'category_id': 62, 'bbox': [252, 197, 46, 99], 'score': 1.0}, {'image_id': 173302, 'category_id': 62, 'bbox': [156, 250, 75, 53], 'score': 0.99}, {'image_id': 173302, 'category_id': 62, 'bbox': [88, 215, 41, 21], 'score': 0.99}, {'image_id': 173302, 'category_id': 62, 'bbox': [219, 196, 32, 72], 'score': 0.96}, {'image_id': 173302, 'category_id': 62, 'bbox': [194, 195, 23, 49], 'score': 0.95}, {'image_id': 173302, 'category_id': 1, 'bbox': [240, 165, 26, 32], 'score': 0.93}, {'image_id': 173302, 'category_id': 82, 'bbox': [119, 152, 27, 72], 'score': 0.81}, {'image_id': 173302, 'category_id': 1, 'bbox': [225, 162, 18, 33], 'score': 0.73}, {'image_id': 173302, 'category_id': 67, 'bbox': [65, 227, 166, 70], 'score': 0.71}, {'image_id': 173302, 'category_id': 62, 'bbox': [66, 220, 24, 16], 'score': 0.54}, {'image_id': 173302, 'category_id': 82, 'bbox': [114, 154, 23, 57], 'score': 0.51}, {'image_id': 173302, 'category_id': 62, 'bbox': [64, 221, 8, 11], 'score': 0.5},

# compare gt and prediction


In [8]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
        
coco_gld = COCO(annFile_val) # coco
if modelv == 2:
    coco_rst = coco_gld.loadRes('./data/results/v2predictions.json')
elif modelv == 3:
    coco_rst = coco_gld.loadRes('./data/results/v3predictions.json')
cocoEval = COCOeval(coco_gld, coco_rst, iouType='bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.28s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.97s).
Accumulating evaluation results...
DONE (t=1.66s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.560
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.508
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.266
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.372
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet